In [6]:
#--------------------------------------------------------------------------------------------------------------------------------
"""
Spectral functions.
"""
heaviside(t) = 0.5 * (sign.(t) .+ 1)
J_ell = w -> real(2*V^2*sqrt.(Complex.(1 .-w.^2))/pi)
J_box = w -> ((V^2)/2)*(heaviside(w .+ 1) .- heaviside(w .- 1))  

#--------------------------------------------------------------------------------------------------------------------------------------
"""
Discretisation mappings and band diagonalisation.
"""
function direct_mapping(f,Nb)
    """
    Implements direct discretisation, using simple trapezium integration.
    """
    samp = 100 # Number of points in mesh.
    y = LinRange(-1,1,Nb+1)
    tsq =  Vector{Float64}(undef,Nb)
    en =  Vector{Float64}(undef,Nb)
    for i =1:Nb
        x = LinRange(y[i],y[i+1],samp)
        Jx =f(x)
        tsq[i] = trapz(x,Jx); 
        en[i] = (1/tsq[i])*trapz(x,x.*Jx);
    end
    
    return [tsq,en] 
end

  
function reaction_mapping(f,Nb)
   
    
    #Define fixed numerical mesh over [-2,2] to capture spectral function and
    # its hilbert transform correctly within [-1,1].
    samp = 1000 # Number of points in mesh.
    x = LinRange(-2,2,samp);
    
    Jx =f(x)  # Evaluate symbolic input function over the grid.
    
    Vsq = zeros(1,Nb)
    en = zeros(1,Nb)
    # Loop over the omega intervals and perform integrations:
    Jcur = Jx; # Current bath spectral function.
    for s=1:Nb
      
      # Simple trapezoid integration for hopping squared and on-site energy:
      Vsq[s] = trapz(x,Jcur); 
      en[s] = (1/Vsq[s])*trapz(x,x.*Jcur);

      Jprev = Jcur;
      JH = imag(hilbert(Jprev)); # Hilbert transform.
      Jcur = (Vsq[s]/(pi^2))*Jprev./(JH.^2+Jprev.^2);
    end
    
    return [Vsq, en]
end
    
    

function band_diag(B,d)
# Band-diagonalize matrix B with a bandwidth of d:
    n = size(B,1); # Assumed to be square.
    U = Diagonal(ones(n,n));
    for k=1:Int(floor(n/d)-1)
        C = B[(k*d+1):n,((k-1)*d+1):(k*d)]; # Extract coupling matrix.
        F = qr(C); # Upper-triangularize.
        blocks = [[Diagonal(ones(k*d,k*d))]; [F.Q']]
        Q = cat(blocks...,dims=(1,2))    # Form full triangularizing unitary.    
        B = Q*B*Q'; # Apply to input matrix to transform for next step.
        U = Q*U; # Save this step's unitary to the full sequence.
    end
    return B,U; # Return the final band-diagonalized matrix.
end;

#------------------------------------------------------------------------------------------------------------------------------------
"""
Useful matrix transformations.
"""

function reflection_diag(A)
    """
    This function takes an N by N matrix, and reflects its elements along the other diagonal (i = N-j). This is used
    to flip the band diagonalised matrices so the modes are in the correct order.
    """
    N = length(A[1,:])
    B = zeros(N,N)
    for i = 0:N-1
        for j = 0:N-1
            B[i+1,j+1] = A[N-j,N-i]
        end
    end
    return B
end

    

    

function U_thermo(N,f_k)
    """
    This unitary maps the energy eigenmode basis to the thermofield basis.
    """
    U_th = zeros(N,N)
    U_th[1,1],U_th[2,2] = 1,1
    b=0
    for i=3:2:N
        b += 1
        U_th[i,i],U_th[i+1,i+1] = sqrt(1-f_k[b]),-sqrt(1-f_k[b])
        U_th[i,i+1],U_th[i+1,i] = sqrt(f_k[b]),sqrt(f_k[b])
    end
    return U_th
end

function U_chain(N,U1,U2)
    """
    This unitary maps the thermofield basis to the tridiagonal thermofield basis. 
    """
    U_tot = zeros(N,N)
    U_tot[1,1],U_tot[2,2] = 1,1
    b1 = 0
    for i=3:2:N
        b2 = 0 # resets the column iteration
        b1 +=1
        for j =3:2:N
            b2 += 1

            U_tot[i,j] = U1[b1,b2]
            U_tot[i+1,j+1] = U2[b1,b2]
        end
    end
    return U_tot
end

#---------------------------------------------------------------------------------------------------------------------------------
"""
Initialisation functions for both the initial state and the hamiltonian.
"""

function initialise_bath_gates(choice,f_k,start,Nb)
    """
    ERROR: The tags of psi are incorrect due to this function.
    """
    stop = start + 2*Nb - 2
    
    if choice == 1
        gates = [(sqrt(f_k[Int((n-1)/2)])*cd[n]*Id[n+1] + sqrt(1-f_k[Int((n-1)/2)])*Id[n]*cd[n+1])/sqrt(2) for n in start:2:stop]; 
    end

    if choice==2 || choice==3
         gates = [cd[n]*Id[n+1] for n in start:2:stop]
    end
    
    return gates
end

function initialise_system_gates(Nbl,Ns)
    start = 2*Nbl+1
    stop = start +2*Ns - 2
    system_gate = [(cd[n]*Id[n+1] + Id[n]*cd[n+1])/sqrt(2) for n in (2*Nbl+1):2:(2*(Ns+Nbl)-1)]
    return system_gate
end



function initialise_psi(gate_list,s)
    n = length(gate_list)
    gates = gate_list[n]
    for j=1:n-1
        for i in reverse(1:length(gate_list[n-j]))
                push!(gates,gate_list[n-j][i])
        end
    end
    vac = productMPS(s, "0");
    ψ = apply(gates,vac;cutoff=1e-15);
    return ψ
end


function initialise_bath(N,J,Nb,β,ϵi,side,disc_choice,choice)
    
    if side =="left"
        n = 1
        start = 1
    elseif side == "right"
        n = Ns
        start = N-2*Nb +1
    else 
        error("neither side chosen")
    end
    
    if disc_choice == 1
        Vsq, ϵb = direct_mapping(J,Nb)
        ind = sortperm(abs.(ϵb.-ϵi[n])) 
        Vsq, ϵb = Vsq[ind], ϵb[ind];  
    
    elseif disc_choice == 2
         Vsq, ϵb = reaction_mapping(J,Nb)
    end
    Vk = sqrt.(Vsq)
    # """
    # Reversing the order of the left bath couplings so the last element corresponds to the bath mode
    # next to the first system mode on the left.
    # """
    if side =="left"
        if choice != 3
            """
            The reason for this condition is that for the tridiagonal case, the bath modes don't need to be put in 
            the right order for the MPO for the band diagonalisation of fill_mat and emp_mat. The order is handled after
            this is done using the reflection_diag function. These arrays don't affect the initialisation of the state
            for the thermofield case so they don't need to be ordered correctly yet.
            """
            Vk = reverse(Vsq)
            ϵb = reverse(ϵb)
        end
    end
    fk = 1 ./(1 .+exp.(β*ϵb))       # Fermi distributions of the left bath modes  
    gates = initialise_bath_gates(choice,fk,start, Nb)
    return Vk,ϵb,fk,gates
end
    

function H_S(ϵi,Ns,Nbl,H_single)
    """
    This function creates the system hamiltonian, excluding the system
    bath couplings.
    """
    terms = OpSum()
    b = 2*Nbl-1
    for i=1:Ns
        b += 2
        terms += ϵi[i],"n",b;
        H_single[b,b] = ϵi[i]
    end
    return terms,H_single
end


function H_bath(Nb,V_k,ϵb,f_k,side,H_single,choice,s)
    #The following code is not optimised, various objects are created multiple times
    #and within each different option there is identical code which doesn't
    #need to be written multiple times.
    """
    One way to make this easier to read is to write a separate function for creating a star geometry hamiltonian. 
    """
    
    ###Create Hamiltonian MPO 
    terms = OpSum()
    ###Create single particle matrix hamiltonian

    N = length(s)
    if side == "left"
        link_ind = 2*Nb + 1
        start = 1
    elseif side == "right"
        link_ind = N-2*Nb - 1
        start = (N-2*Nb +1)
    else 
        error("start input is invalid")
    end
    stop = start +2*Nb - 2
    
    
    b = 0
    if choice ==1
        print("negative ancilla H = -1, no ancilla H = 0, positive ancilla H = 1")
        HA_choice = parse(Int,readline()) 
        model =  "E basis,"
        if HA_choice ==1
            model = model*"Ha=Hb"
        end
        if HA_choice ==0
            model = model*"Ha=0"
        end
        if HA_choice ==-1
            model = model*"Ha=-1"
        end
        for j=start:2:stop
            b += 1
            terms += ϵb[b],"n",j                                   # bath mode self energy
            H_single[j,j] = ϵb[b]

            terms += HA_choice*ϵb[b],"n",j+1                       # ancilla bath mode self energy
            H_single[j+1,j+1] =  HA_choice*ϵb[b]

            terms += V_k[b],"Cdag",j,"C",link_ind                        #hopping from system to kth f mode 
            H_single[j,link_ind] = V_k[b]

            terms += conj(V_k[b]),"Cdag",link_ind,"C",j                   #hopping from kth f mode to system 
            H_single[link_ind,j] = conj(V_k[b])
        end    
    end

    if choice==2
        model = "thermofield basis"
        for j=start:2:stop
            b += 1
            terms += ϵb[b],"n",j                                   # filled mode self energy
            H_single[j,j] = ϵb[b]

            terms += ϵb[b],"n",j+1                                 # empty mode self energy
            H_single[j+1,j+1] =  ϵb[b]

            terms += V_k[b]*sqrt(f_k[b]),"Cdag",j,"C",link_ind            #hopping from system to kth f mode
            H_single[j,link_ind] = V_k[b]*sqrt(f_k[b])

            terms += conj(V_k[b])*sqrt(f_k[b]),"Cdag",link_ind,"C",j      #hopping from kth f mode to system 
            H_single[link_ind,j] = conj(V_k[b])*sqrt(f_k[b])

            terms += V_k[b]*sqrt(1-f_k[b]),"Cdag",j+1,"C",link_ind        #coupling from system to kth e mode
            H_single[j+1,link_ind] =  V_k[b]*sqrt(1-f_k[b])

            terms += conj(V_k[b])*sqrt(1-f_k[b]),"Cdag",link_ind,"C",j+1  #coupling from kth e mode to system
            H_single[link_ind,j+1] =  conj(V_k[b])*sqrt(1-f_k[b])
        end
    end

    if choice ==3
        model = "thermofield+tridiag"
        fill_mat = zeros(Nb+1,Nb+1)
        emp_mat = zeros(Nb+1,Nb+1)
        ### Same terms as for choice 2, inputed as a matrix rather than an MPO. 
        for j=1:Nb
            ###This loop creates two (Nb+1) x (Nb+1) matrices, one includes the couplings and self energies of the 
            ###filled modes and the system, the other the empty modes and the system. These are then tridiagonalised 
            ###separately in the next loop, and their elements are used to construct the MPO for the hamiltonian in this new, tridiagonal basis. 
            
            """
            Pretty sure the first entry doesn't effect the diagonalisation so I set it to zero for both.
            """
            fill_mat[1,1],emp_mat[1,1] = 0, 0

            fill_mat[j+1,j+1],emp_mat[j+1,j+1] = ϵb[j],ϵb[j]

            fill_mat[j+1,1] = V_k[j]*sqrt(f_k[j])

            fill_mat[1,j+1] = conj(V_k[j])*sqrt(f_k[j])

            emp_mat[j+1,1] = V_k[j]*sqrt(1-f_k[j])

            emp_mat[1,j+1] = conj(V_k[j])*sqrt(1-f_k[j])
        end
   
        fill_mat,Uf = band_diag(fill_mat,1)
        emp_mat,Ue = band_diag(emp_mat,1)

        ###discarding the system terms as the system isn't mixed in this transformation.
        Uf = Uf[2:Nb+1,2:Nb+1] 
        Ue = Ue[2:Nb+1,2:Nb+1]
        
        b = 1
        if side == "left"
            ##flipping the matrices so the modes closest to the system
            ##are further along the matrix (in terms of i,j)
            fill_mat = reflection_diag(fill_mat)
            emp_mat = reflection_diag(emp_mat)
            #As the system mode is now at the end of 
            #these matrices, we don't skip over this at the start of the next loop
            b = 0 
        end
        
        for j=start:2:stop
            b += 1
            terms += fill_mat[b,b],"n",j
            H_single[j,j] = fill_mat[b,b]

            terms += emp_mat[b,b],"n",j+1
            H_single[j+1,j+1] = emp_mat[b,b]
            if side =="right"
                terms += fill_mat[b-1,b],"Cdag",j-2,"C",j
                H_single[j-2,j] = fill_mat[b-1,b]

                terms += fill_mat[b,b-1],"Cdag",j,"C",j-2
                H_single[j,j-2] = fill_mat[b,b-1]
                
                if j == start 
                    terms += emp_mat[b-1,b],"Cdag",j-2,"C",j+1
                    H_single[j-2,j+1] = emp_mat[b-1,b]

                    terms += emp_mat[b,b-1],"Cdag",j+1,"C",j-2
                    H_single[j+1,j-2] = emp_mat[b,b-1]    
                else             
                    terms += emp_mat[b-1,b],"Cdag",j-1,"C",j+1
                    H_single[j-1,j+1] = emp_mat[b-1,b]

                    terms += emp_mat[b,b-1],"Cdag",j+1,"C",j-1
                    H_single[j+1,j-1] = emp_mat[b,b-1]
                end
            else side =="left"      
                """
                CHECK THAT THIS ORDER IS CORRECT, I.E WHETHER THE J AND J+2 
                SHOULD BE SWITCHED
                """
                terms += fill_mat[b,b+1],"Cdag",j,"C",j+2
                H_single[j,j+2] = fill_mat[b,b+1]

                terms += fill_mat[b+1,b],"Cdag",j+2,"C",j
                H_single[j+2,j] = fill_mat[b+1,b]
                if j == stop
                    terms += emp_mat[b,b+1],"Cdag",j+1,"C",j+2
                    H_single[j+1,j+2] = emp_mat[b,b+1]
                    
                    terms += emp_mat[b+1,b],"Cdag",j+2,"C",j+1
                    H_single[j+2,j+1] = emp_mat[b+1,b]
                else
                    terms += emp_mat[b,b+1],"Cdag",j+1,"C",j+3
                    H_single[j+1,j+3] = emp_mat[b,b+1]
                    
                    terms += emp_mat[b+1,b],"Cdag",j+3,"C",j+1
                    H_single[j+3,j+1] = emp_mat[b+1,b]
                end
            end
        end
    end

    return  terms,H_single
end


H_ (generic function with 1 method)

H_bath